In [2]:
from zipline import run_algorithm
from zipline.api import order_target_percent, symbol
from datetime import datetime
import pytz
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
def initialize(context):
    context.stock = symbol('AAPL')
    context.index_average_window = 100
    # Set zipline benchmark to false
    context.set_benchmark(False)
    
def handle_data(context, data):
    equities_hist = data.history(context.stock, 'close', context.index_average_window, '1d')
    if equities_hist[-1] > equities_hist.mean():
        stock_weight = 1.0
    else:
        stock_weight = 0.0
    
    # Place order
    order_target_percent(context.stock, stock_weight)
    
def analyze(context, perf):
    fig = plt.figure(figsize=(12, 8))
    
    ax = fig.add_subplot(311)
    ax.set_title('Strategy results')
    ax.semilogy(perf['portfolio_value'], linestyle='-', label='Equity Curve', linewidth=3)
    ax.legend()
    ax.grid(False)
    
    ax2 = fig.add_subplot(312)
    ax2.plot(perf['gross_leverage'], label='Exposure', linestyle='-', linewidth=1)
    ax2.legend()
    ax2.grid(True)
    
    ax3 = fig.add_subplot(313)
    ax3.plot(perf['returns'], label='Returns', linestyle='-.', linewidth=1)
    ax3.legend()
    ax3.grid(True)

# Set start date and end date    
#start_date = datetime(2012, 1, 1, tzinfo=pytz.UTC)
#end_date = datetime(2015, 12, 31, tzinfo=pytz.UTC)

start_date = pd.Timestamp('2002-1-1', tz='utc')
end_date = pd.Timestamp('2017-12-31', tz='utc')
#type(start_date)

# Fire off the backtest
results = run_algorithm(
    start=start_date, 
    end=end_date, 
    initialize=initialize, 
    analyze=analyze, 
    handle_data=handle_data, 
    capital_base=10000, 
    data_frequency = 'daily', bundle='quandl' 
) 